<H1> Coursera Capstone- Toronto Neighbourhood Assignment

This notebook is in response to Coursera Capstone Week 3 assignment. It will utilize Foursquare API and newly acquired skills from the course.

In [1]:
#getting the imports out of the way
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

#!conda install -c anaconda beautifulsoup4
from bs4 import BeautifulSoup
import requests
import lxml
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


<H2> Scraping Website

In [4]:
#linking and scraping website
r= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table= soup.find('table', attrs={'class':'wikitable sortable'})

headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

rows=table.findAll('tr')
rows=rows[1:len(rows)]

for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

#Creating the Data Frame
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

df=df.drop(df[(df.Borough== "Not assigned")].index)
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

df.Neighbourhood.fillna(df.Borough, inplace=True)
df=df.drop_duplicates()

df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

Toronto = pd.DataFrame({'Postcode':df.Postcode.unique()})
Toronto['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in Toronto.iterrows())
Toronto['Neighbourhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in Toronto.iterrows())
Toronto['Neighbourhood']=Toronto['Neighbourhood'].apply(lambda x: ', '.join(x))
Toronto.dtypes

Toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park


In [3]:
table_columns=['Postalcode', 'Borough', 'Neighborhood']
toronto = pd.DataFrame(columns = table_columns)
content = soup.find('div', class_='mw-parser-output')
table= content.table.tbody
postcode=0
borough=0
neighborhood=0

for tr in table.find_all('tr'):
    i=0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i= i+1
        elif i==1:
            borough = td.text
            i=i+1
        elif i==2:
            neighborhood = td.text.strip('\n').replace(']','')
        toronto = toronto.append({'Postalcode': postcode, 'Borough': borough, 'Neighborhood':neighborhood}, ignore_index=True)

toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!=0]
toronto.reset_index(drop=True, inplace=True)
i=0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2]= toronto.iloc[i][1]
        i=i+1
df= toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(12)

,Postalcode,Borough,Neighborhood
0,M1B,Etobicoke,Islington Avenue\n
1,M1B,Scarborough,"Islington Avenue\n, Rouge\n, Rouge\n, Rouge\n,..."
2,M1C,Etobicoke,West Deane Park\n
3,M1C,Scarborough,"West Deane Park\n, Highland Creek\n, Highland ..."
4,M1E,Etobicoke,Old Burnhamthorpe\n
5,M1E,Scarborough,"Old Burnhamthorpe\n, Guildwood\n, Guildwood\n,..."
6,M1G,Scarborough,"Not assigned\n, Woburn\n"
7,M1H,Scarborough,"Not assigned\n, Cedarbrae\n"
8,M1J,Scarborough,"Not assigned\n, Scarborough Village\n"
9,M1K,Scarborough,"Not assigned\n, East Birchmount Park\n, East B..."
